# Análise do bancos de dados construído pelo Crawler Mercado Livre

## Importação de bibliotecas

In [590]:
import os
from sqlalchemy import create_engine
import pandas as pd

## Carregando variáveis de ambiente e lendo o banco

In [591]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [592]:
DATABASE_URL = os.environ.get("DATABASE_URL").replace("postgres://", "postgresql://", 1)
engine = create_engine(DATABASE_URL)

In [593]:
df = pd.read_sql("SELECT * FROM mercado_livre_offers", con=engine)
pd.options.mode.chained_assignment = None 

## Análise inicial

Amostra do começo e do final do dataset

In [594]:
df.head()

,name,categories,best_seller,offer_of_the_day,discount,old_price,new_price,date
0,Samsung Galaxy A32 Dual SIM 128 GB branco 4 GB...,"[Celulares e Telefones, Celulares e Smartphone...","[MAIS VENDIDO, 17º em Celulares e Smartphones ...",True,15,1.799,1512,2022-04-12
1,Philco Hit P10 Dual SIM 128 GB dourado 4 GB RAM,"[Celulares e Telefones, Celulares e Smartphone...",[],True,29,1.099,779,2022-04-12
2,Cloro Granulado 3 Em 1 Multi Ação 10kg - Genco,"[..., ..., ..., Limpeza e Manutenção, Acessóri...","[MAIS VENDIDO, 1º em Kits Teste Ph e Cloro]",False,5,301,286.14,2022-04-12
3,Notebook Samsung Chromebook XE310XBA cinza 11....,"[Informática, Portáteis e Acessórios, Notebook...",[],True,18,2.299,1879,2022-04-12
4,Multilaser F Dual SIM 32 GB café 1 GB RAM,"[Celulares e Telefones, Celulares e Smartphone...",[],True,9,459,414.90,2022-04-12


In [595]:
df.tail()

,name,categories,best_seller,offer_of_the_day,discount,old_price,new_price,date
9308,Alternador Peugeot 206 307 407 607 Partner Ber...,"[..., Peças de Carros e Caminhonetes, Ignição,...",[],False,5,889,845.49,2022-04-20
9309,Colchão Casal Inflável + Travesseiro Incorpora...,"[..., ..., Equipamento para Camping, Sacos de ...",[],False,16,328,276.27,2022-04-20
9310,Mini Adaptador Conversor Hdmi P/ Vga Transmite...,"[Eletrônicos, Áudio e Vídeo, Acessórios para Á...","[MAIS VENDIDO, 4º em Conversores de Aúdio e Ví...",False,5,23,22.04,2022-04-20
9311,Luz Led Neon Cores Ambiente Interno Automotivo...,"[Acessórios para Veículos, Tuning, Iluminação,...","[MAIS VENDIDO, 5º em Neon]",False,21,78,62.33,2022-04-20
9312,Fogão de piso Itatiaia Waves 6Q gás engarra...,"[Eletrodomésticos, Fornos e Fogões, Fogões, It...","[MAIS VENDIDO, 6º em Fogões]",True,14,1.049,899,2022-04-20


Verificando o tamanho do dataset, suas colunas e tipos de dados

In [596]:
print(df.shape, '\n', '', '\n', df.columns, '\n', '', '\n', df.dtypes)

(9313, 8) 
  
 Index(['name', 'categories', 'best_seller', ' offer_of_the_day', 'discount',
       'old_price', 'new_price', 'date'],
      dtype='object') 
  
 name                         object
categories                   object
best_seller                  object
 offer_of_the_day              bool
discount                     object
old_price                    object
new_price                    object
date                 datetime64[ns]
dtype: object


Verificando valores nulos

In [597]:
df.isnull().sum()

name                   0
categories             0
best_seller            0
 offer_of_the_day      0
discount             293
old_price             45
new_price             75
date                   0
dtype: int64

Aparentemente, os valores nulos estão nas colunas 'discount', 'old_price' e 'new_price', no entanto, ao vermos uma amostra do dataset, encontramos listas vazias na coluna 'best_seller'

## Tratando os dados nulos e erros de cálculos

### Removendo valores

Vamos verificar os valores nulos nas colunas 'new_price' e 'old_price'. Caso os valores de ambas as colunas sejam nulos ou os valores de 'discount' e 'old_price' ou 'new_price', então não é possível calcular qualquer destes campos

In [598]:
df.loc[df['old_price'].isnull() & df['new_price'].isnull()]

,name,categories,best_seller,offer_of_the_day,discount,old_price,new_price,date
873,LG K62+ Dual SIM 128 GB white 4 GB RAM,[],[],False,None,None,None,2022-04-12
2291,Bebida Vegetal Original Orgânico A Tal da Cast...,[],[],False,None,None,None,2022-04-13
2514,Bebida Vegetal Amêndoa A Tal da Castanha 1 Litro,[],[],False,None,None,None,2022-04-13
2591,Talco Leite de Rosas Barla Perfumado em frasco...,[],[],False,None,None,None,2022-04-13
2718,"Som automotivo Svart S500 com USB, bluetooth e...",[],[],False,None,None,None,2022-04-14
2977,Chips de Pipoca Manteiga Roots To Go Pacote 35g,[],[],False,None,None,None,2022-04-13
3602,Fórmula infantil em pó Nestlé Nan Supreme 1 e...,[],[],False,None,None,None,2022-04-14
3656,Samsung Galaxy Z Flip3 5G 128 GB cream 8 GB RAM,[],[],False,None,None,None,2022-04-13
3661,Samsung Galaxy Note20 Ultra 5G Dual SIM 256 GB...,[],[],False,None,None,None,2022-04-14
3893,Cafeteira Cadence O’clock CAF205 semi automáti...,[],[],False,None,None,None,2022-04-19


Vemos que neste caso todos os campos estão sem valores, então vamos remover todos os registros com valores nulos.

In [599]:
df.drop(df.loc[df['old_price'].isnull() & df['new_price'].isnull()].index, inplace=True)

Ainda verificando outros valores nulos

In [600]:
df.loc[df['new_price'].isnull()]

,name,categories,best_seller,offer_of_the_day,discount,old_price,new_price,date
1705,Controle joystick Leboss LB-2121/S preto,[],[],False,None,35,None,2022-04-12
2034,Martelete DeWalt D25133K amarelo e preto frequ...,[],[],False,None,760,None,2022-04-15
2632,Torneira de cozinha monocomando Pingoo.Casa En...,[],[],False,None,349,None,2022-04-13
2740,No break JBR 800 800VA entrada e saída de 120V...,[],[],False,None,469,None,2022-04-13
3061,"Lixadeira orbital Mondial FLO-2 preta, verm...",[],[],False,None,182,None,2022-04-13
3301,Mouse sem fio Logitech Pebble M350 cinza-azulado,[],[],False,None,99,None,2022-04-13
3470,Teclado sem fio Logitech K230 QWERTY português...,[],[],False,None,116,None,2022-04-13
3588,Chivas Regal Blended Extra 13 Reino Unido 750 mL,[],[],False,None,131,None,2022-04-14
3757,Caminhonete de controle remoto Importway BW026...,[],[],False,None,102,None,2022-04-14
3958,Alto-falante Grasep D-P13 com bluetooth inglat...,[],[],False,None,94,None,2022-04-13


Neste caso, apenas os campos de 'old_price' possuem valores, tornando-se também inviável para qualquer cálculo

In [601]:
df.drop(df.loc[df['new_price'].isnull()].index, inplace=True)

Vamos verificar novamente os valores nulos do nosso dataset

In [602]:
df.isnull().sum()

name                   0
categories             0
best_seller            0
 offer_of_the_day      0
discount             218
old_price              0
new_price              0
date                   0
dtype: int64

### Transformando o tipo de dados

Agora vamos transformar os tipos de dados dos campos 'new_price' e 'old_price' para float, para podermos calcular os valores nulos ainda restante na coluna 'discount'.

In [603]:
# Precisei usar o replace pois alguns valores como 1.100 foi convertido para 1.1 em float ao invés de 1100
df['old_price'] = df['old_price'].apply(lambda x: int(x.replace('.', '')))
df['new_price'] = df['new_price'].astype('float')

Vamos transformar também a coluna 'discount' em valores inteiros

In [604]:
df['discount'] = df['discount'].apply(lambda x: int(x) if x != None else x)

### Corrigindo valores não nulos de 'new_price'

Vemos abaixo que possuimos erros de conta em 'new_price', sendo assim, vamos corrigi-los. 

1) Vamos primeiramente identificar estas linhas

In [605]:
df.loc[(df['old_price'] < df['new_price']) & df['discount'].notnull()]

,name,categories,best_seller,offer_of_the_day,discount,old_price,new_price,date
470,Sabonete Infantil Em Barra Cremer Loção Hidrat...,"[Beleza e Cuidado Pessoal, Higiene Pessoal, Sa...",[],False,24.0,3,3.03,2022-04-12
1155,Lençol Descartavel Papel Hospitalar Maca 70x50...,"[Beleza e Cuidado Pessoal, Higiene Pessoal, Le...",[],False,5.0,10,10.35,2022-04-12
1551,Leite Longa Vida Semidesnatado com tampa Itala...,"[Alimentos e Bebidas, Frescos, Laticínios, Lei...",[],False,6.0,4,4.12,2022-04-13
1560,Refrigerante Coca-Cola Garrafa 250ml,"[Alimentos e Bebidas, Bebidas, Refrigerantes]","[MAIS VENDIDO, 6º em Refrigerantes]",False,14.0,3,3.34,2022-04-13
1696,Cabo Otg Usb Tablet Celular Motorola Samsung P...,"[Celulares e Telefones, Acessórios para Celula...","[MAIS VENDIDO, 1º em USB It-Blue]",True,5.0,13,13.19,2022-04-13
...,...,...,...,...,...,...,...,...
8759,50 Escovinha Alongamento Cílios Sobrancelha,"[Beleza e Cuidado Pessoal, Maquiagem, Olhos, C...",[],False,5.0,18,18.04,2022-04-20
8762,Biscoito Recheio Leite Passatempo Pacote 130g,"[Alimentos e Bebidas, Mercearia, Biscoitos]","[MAIS VENDIDO, 15º em Biscoitos]",False,10.0,2,2.24,2022-04-20
8843,Marshmallow Fini Torção 250g,"[Alimentos e Bebidas, Mercearia, Doces e Choco...",[],False,7.0,7,7.43,2022-04-20
8942,Ralo Tampa Grelha Inox Redondo 10 Cm Abre E Fe...,"[Casa, Móveis e Decoração, Banheiros, Acessóri...","[MAIS VENDIDO, 6º em Grelhas]",False,7.0,11,11.15,2022-04-20


2) Vamos adicionar este dataframe a uma variável e remove-lo do dataset original

In [606]:
df_fix_new_price = df.loc[(df['old_price'] < df['new_price']) & df['discount'].notnull()]
df.drop(df.loc[(df['old_price'] < df['new_price']) & df['discount'].notnull()].index, inplace=True)

3) Agora vamos corrigir os valores de 'new_price'

In [607]:
df_fix_new_price['new_price'] = df_fix_new_price['old_price']*(100-df_fix_new_price['discount'])/100

4. Agora vamos adicionar este dataframe ao dataset original

In [608]:
df = pd.concat([df, df_fix_new_price])

### Corrigindo valores nulos de discount

Vamos realiazar as mesmas operações, separar, remover, corrigir e adicionar os valores novamente no dataframe

In [609]:
df_discount_null = df.loc[df['discount'].isnull()]
df.drop(df.loc[df['discount'].isnull()].index, inplace=True)

In [610]:
df_discount_null['discount'] = round((1-(df_discount_null['new_price']/df_discount_null['old_price']))*100, 2)

In [611]:
df_discount_null.head()

,name,categories,best_seller,offer_of_the_day,discount,old_price,new_price,date
70,Micro-ondas Electrolux MTD30 branco 20L 127V,"[Eletrodomésticos, Fornos e Fogões, Microondas...","[MAIS VENDIDO, 19º em Microondas]",True,3.35,537,519.00,2022-04-12
257,Placa Interface Geladeira Electrolux Df50 6480...,"[Eletrodomésticos, Refrigeração, Peças e Acess...","[MAIS VENDIDO, 2º em Placas para Geladeiras]",False,-0.48,199,199.95,2022-04-12
297,"Cabo Flexível 10,0mm Preto 100m - Sil","[Construção, Energia, Cabos e Acessórios, Filt...",[],False,4.91,939,892.91,2022-04-12
318,Maleta Organizadora 34 Esmaltes Manicure+porta...,"[Beleza e Cuidado Pessoal, Manicure e Pedicure...","[MAIS VENDIDO, 1º em Esmaltes Dompel]",True,3.44,61,58.90,2022-04-12
343,Tablet Samsung Galaxy Tab S6 Lite SM-P615 10....,"[Informática, Tablets e Acessórios, Tablets, S...","[MAIS VENDIDO, 1º em Tablets]",False,4.80,2499,2379.00,2022-04-12


Alguns valores de 'discount' ficaram negativos devido aos valores de new_price serem maior do que old_price. Vamos remover estes registros visto que não faz sentido um desconto negativo, aumentando o preço final.

In [612]:
df_discount_null.loc[df_discount_null['discount'] < 0].head()

,name,categories,best_seller,offer_of_the_day,discount,old_price,new_price,date
257,Placa Interface Geladeira Electrolux Df50 6480...,"[Eletrodomésticos, Refrigeração, Peças e Acess...","[MAIS VENDIDO, 2º em Placas para Geladeiras]",False,-0.48,199,199.95,2022-04-12
380,Lixa De Polir Unhas Bloco Branco 4 Faces Brilh...,"[Beleza e Cuidado Pessoal, Manicure e Pedicure...","[MAIS VENDIDO, 6º em Lixas de Unhas Dompel]",True,-10.57,7,7.74,2022-04-12
435,Suporte Lixa Pés Anatômica Lixa Descartável Re...,"[Beleza e Cuidado Pessoal, Manicure e Pedicure...","[MAIS VENDIDO, 20º em Lixas de Unhas Dompel]",True,-11.86,7,7.83,2022-04-12
998,Acabamento Quadrado Para Registro Padrão Deca ...,"[Construção, Mobiliário para Banheiros, Tornei...","[MAIS VENDIDO, 17º em Torneiras Convencionais]",False,-1.96,46,46.90,2022-04-12
1087,Driver D220ti Trio Titanio - 90 Watts Rms Jb...,"[Acessórios para Veículos, Som Automotivo, Dri...",[],False,-0.18,248,248.45,2022-04-15


In [613]:
df_discount_null.drop(df_discount_null.loc[df_discount_null['discount'] < 0].index, inplace=True)

Agora vamos adicionar este dataframe ao dataset original novamente

In [614]:
df = pd.concat([df, df_discount_null])

Verificando se ainda há valores nulos

In [615]:
df.isnull().sum()

name                 0
categories           0
best_seller          0
 offer_of_the_day    0
discount             0
old_price            0
new_price            0
date                 0
dtype: int64

### Calculando valor real de 'discount'

Fazendo uma conta simples, podemos perceber que o valor de 'discount' mostrado no dataset não é valor real de desconto do 'old_price' para o 'new_price'.

In [616]:
df.head(1)

,name,categories,best_seller,offer_of_the_day,discount,old_price,new_price,date
0,Samsung Galaxy A32 Dual SIM 128 GB branco 4 GB...,"[Celulares e Telefones, Celulares e Smartphone...","[MAIS VENDIDO, 17º em Celulares e Smartphones ...",True,15.0,1799,1512.0,2022-04-12


Vamos calcular o desconto deste primeiro produto

In [617]:
(1-1512/1799)*100

15.953307392996107

Vemos que temos uma diferença de quase 1% entre o valor no dataset e o valor real de desconto. Desta maneira, vamos recalcular todos os descontos dos produtos.

In [618]:
df['discount'] = round((1-(df['new_price']/df['old_price']))*100, 2)

In [619]:
df.head()

,name,categories,best_seller,offer_of_the_day,discount,old_price,new_price,date
0,Samsung Galaxy A32 Dual SIM 128 GB branco 4 GB...,"[Celulares e Telefones, Celulares e Smartphone...","[MAIS VENDIDO, 17º em Celulares e Smartphones ...",True,15.95,1799,1512.00,2022-04-12
1,Philco Hit P10 Dual SIM 128 GB dourado 4 GB RAM,"[Celulares e Telefones, Celulares e Smartphone...",[],True,29.12,1099,779.00,2022-04-12
2,Cloro Granulado 3 Em 1 Multi Ação 10kg - Genco,"[..., ..., ..., Limpeza e Manutenção, Acessóri...","[MAIS VENDIDO, 1º em Kits Teste Ph e Cloro]",False,4.94,301,286.14,2022-04-12
3,Notebook Samsung Chromebook XE310XBA cinza 11....,"[Informática, Portáteis e Acessórios, Notebook...",[],True,18.27,2299,1879.00,2022-04-12
4,Multilaser F Dual SIM 32 GB café 1 GB RAM,"[Celulares e Telefones, Celulares e Smartphone...",[],True,9.61,459,414.90,2022-04-12


Agora vamos verificar se não temos nenhum valor negativo de desconto

In [620]:
df.loc[df['discount'] < 0]

,name,categories,best_seller,offer_of_the_day,discount,old_price,new_price,date


## Definindo a categoria dos produtos

Como podemos ver abaixo, cada produto se encaixa em diversas categorias, as quais vão se especificando mais e mais dentro do site do mercado livre. Deste modo, vamos criar duas novas colunas:

1. Coluna 'category', a qual representa uma categoria mais geral do produto
2. Coluna 'specific_category', a qual representa uma categoria específica do produto com base na coluna 'best_seller'

In [621]:
df['categories'].head()

0    [Celulares e Telefones, Celulares e Smartphone...
1    [Celulares e Telefones, Celulares e Smartphone...
2    [..., ..., ..., Limpeza e Manutenção, Acessóri...
3    [Informática, Portáteis e Acessórios, Notebook...
4    [Celulares e Telefones, Celulares e Smartphone...
Name: categories, dtype: object

In [622]:
df['best_seller'].head()

0    [MAIS VENDIDO, 17º em Celulares e Smartphones ...
1                                                   []
2          [MAIS VENDIDO, 1º em Kits Teste Ph e Cloro]
3                                                   []
4                                                   []
Name: best_seller, dtype: object

### Definindo coluna categoria específica

Podemos notar que em 'best_seller' temos uma lista com dois valores. O primeiro se trata de anunciar que aquele produto estava entre os mais vendidos daquele dia, e o segundo é a categoria que se encontrava. Deste modo, vamos definir a categoria específica com base no segundo valor desta lista.

Precisamos de uma função que faça igual o procedimento abaixo: 

In [623]:
categoria = ['MAIS VENDIDO','14º em Máquinas de Cortar Cabelo Wmark']
' '.join(categoria[1].split()[2:])

'Máquinas de Cortar Cabelo Wmark'

In [624]:
df['specific_category'] = df['best_seller'].apply(lambda x: ' '.join(x[1].split()[2:]) if x != [] else '')

### Resultado e análise da coluna specific_category

Vamos ver o dataframe com a nova coluna

In [625]:
df.head(10)

,name,categories,best_seller,offer_of_the_day,discount,old_price,new_price,date,specific_category
0,Samsung Galaxy A32 Dual SIM 128 GB branco 4 GB...,"[Celulares e Telefones, Celulares e Smartphone...","[MAIS VENDIDO, 17º em Celulares e Smartphones ...",True,15.95,1799,1512.00,2022-04-12,Celulares e Smartphones Samsung
1,Philco Hit P10 Dual SIM 128 GB dourado 4 GB RAM,"[Celulares e Telefones, Celulares e Smartphone...",[],True,29.12,1099,779.00,2022-04-12,
2,Cloro Granulado 3 Em 1 Multi Ação 10kg - Genco,"[..., ..., ..., Limpeza e Manutenção, Acessóri...","[MAIS VENDIDO, 1º em Kits Teste Ph e Cloro]",False,4.94,301,286.14,2022-04-12,Kits Teste Ph e Cloro
3,Notebook Samsung Chromebook XE310XBA cinza 11....,"[Informática, Portáteis e Acessórios, Notebook...",[],True,18.27,2299,1879.00,2022-04-12,
4,Multilaser F Dual SIM 32 GB café 1 GB RAM,"[Celulares e Telefones, Celulares e Smartphone...",[],True,9.61,459,414.90,2022-04-12,
5,Módulo Taramps Ts400 Digital 400w Rms 4 Canais...,"[Acessórios para Veículos, Som Automotivo, Mód...","[MAIS VENDIDO, 1º em Módulos Amplificadores]",False,10.54,257,229.90,2022-04-12,Módulos Amplificadores
6,Moto G31 Dual SIM 128 GB grafite 4 GB RAM,"[Celulares e Telefones, Celulares e Smartphone...",[],True,15.60,1699,1434.00,2022-04-12,
7,Samsung Galaxy A71 Dual SIM 128 GB azul 6 GB RAM,"[Celulares e Telefones, Celulares e Smartphone...",[],False,47.16,3499,1849.00,2022-04-12,
8,"Fritadeira Air Fry Inox Bfr11pi 3,5 Litros Bri...","[..., Pequenos Eletrodomésticos, Para a Cozinh...","[MAIS VENDIDO, 1º em De Ar]",False,47.76,689,359.90,2022-04-12,De Ar
9,Moto E20 Dual SIM 32 GB azul 2 GB RAM,"[Celulares e Telefones, Celulares e Smartphone...","[MAIS VENDIDO, 5º em Celulares e Smartphones]",True,28.21,1099,789.00,2022-04-12,Celulares e Smartphones


Vamos ver quais e quantas categorias específicas nós temos

In [626]:
df['specific_category'].value_counts()

                                    5518
Celulares e Smartphones               18
Ar Condicionado                       14
Televisores                           13
Celulares e Smartphones Motorola      11
                                    ... 
Tupias                                 1
Extintores                             1
Termômetros Culinários                 1
Cabos de Áudio e Vídeo Vention         1
Kits de Irrigação                      1
Name: specific_category, Length: 1899, dtype: int64

Tirando os valores '', temos 1898 categorias específicas. Contudo, ainda temos 5511 valores vazios. Vamos tentar preenche-los.

### Preenchendo valores vazios das categorias específicas

Vamos agora preencher os valores vazios da coluna 'specific_category' procurando estes valores dentro da lista que está na coluna 'categories'

In [627]:
# Criando uma lista com os valores a serem procurados

specific = df['specific_category'].value_counts().index
specific = specific.to_list()
specific[1:]

['Celulares e Smartphones',
 'Ar Condicionado',
 'Televisores',
 'Celulares e Smartphones Motorola',
 'Descartáveis',
 'Vinhos',
 'Monitores',
 'Fontes Chaveadas',
 'Elétricos',
 'Kit de Chaves',
 'Cervejas',
 'Torneiras Convencionais',
 'Desodorantes',
 'Brocas',
 'Andadores',
 'Shampoos e Condicionadores',
 'Suplementos Alimentares Integralmédica',
 'Travesseiros',
 'Protetores Solares',
 'Divisores de Frequência',
 'Computadores',
 'Extensores e Elásticos',
 'Lubrificantes',
 'Amaciante',
 'Sem Tela',
 'Lixadeiras',
 'Joysticks',
 'Suplementos',
 'Secadores de Cabelo',
 'Suportes',
 'Baterias',
 'Tapete Eva',
 'Alarmes e Sensores',
 'Para TV',
 'Televisores Philips',
 'Jogos e Kits de Toalhas',
 'Cabides',
 'Grelhas',
 'Grupos e Kits',
 'Kit de Chaves Sata',
 'Receptores para TV',
 'Controles Remotos',
 'Luvas de Procedimentos',
 'Tablets',
 'Módulos Amplificadores',
 'Rede de Proteção',
 'Balanças',
 'Digitais',
 'Notebooks',
 'Placas Acústicas',
 'Líquido',
 'Preservativos',
 'Col

Função para procurar valores na lista e preencher os valores nulos de 'specific_category'

In [628]:
def fill_specific_category_empty_values(specific_category, categories, specific):
    if specific_category == '':
        for cat in specific:
            if cat in categories:
                return cat
    elif specific_category != '':
        return specific_category
    else:
        return ''

In [629]:
df['specific_category'] = df.apply(lambda row: fill_specific_category_empty_values(row['specific_category'], row['categories'], specific), axis=1)

In [646]:
df['specific_category'].value_counts()

Ferramentas                     229
Celulares e Smartphones         152
Livros Fisicos                  149
Tratamentos com o Cabelo         98
Acessórios                       84
                               ... 
Surdos                            1
Junta de Homocinetica Nakata      1
Bolas                             1
Espelhos                          1
Vinhos Reservado                  1
Name: specific_category, Length: 1898, dtype: int64

In [641]:
df.isnull().sum()

name                    0
categories              0
best_seller             0
 offer_of_the_day       0
discount                3
old_price               0
new_price               0
date                    0
specific_category    1357
dtype: int64

Podemos ver que diversos valores foram encontrados, porém ainda temos 1357 em branco.

### Criando coluna 'category' com base na coluna 'categories'

In [649]:
df['category'] = df['categories'].apply(lambda x: x[0])

In [650]:
df

,name,categories,best_seller,offer_of_the_day,discount,old_price,new_price,date,specific_category,category
0,Samsung Galaxy A32 Dual SIM 128 GB branco 4 GB...,"[Celulares e Telefones, Celulares e Smartphone...","[MAIS VENDIDO, 17º em Celulares e Smartphones ...",True,15.95,1799,1512.00,2022-04-12,Celulares e Smartphones Samsung,Celulares e Telefones
1,Philco Hit P10 Dual SIM 128 GB dourado 4 GB RAM,"[Celulares e Telefones, Celulares e Smartphone...",[],True,29.12,1099,779.00,2022-04-12,Celulares e Smartphones,Celulares e Telefones
2,Cloro Granulado 3 Em 1 Multi Ação 10kg - Genco,"[..., ..., ..., Limpeza e Manutenção, Acessóri...","[MAIS VENDIDO, 1º em Kits Teste Ph e Cloro]",False,4.94,301,286.14,2022-04-12,Kits Teste Ph e Cloro,...
3,Notebook Samsung Chromebook XE310XBA cinza 11....,"[Informática, Portáteis e Acessórios, Notebook...",[],True,18.27,2299,1879.00,2022-04-12,Notebooks,Informática
4,Multilaser F Dual SIM 32 GB café 1 GB RAM,"[Celulares e Telefones, Celulares e Smartphone...",[],True,9.61,459,414.90,2022-04-12,Celulares e Smartphones,Celulares e Telefones
...,...,...,...,...,...,...,...,...,...,...
8814,5l Grande Capacidade Ao Ar Livre Ciclismo Moch...,"[..., Ciclismo, Acessórios , Transporte e Carg...",[],False,4.94,133,126.43,2022-04-20,None,...
8832,Cajon + Bag Case,"[Instrumentos Musicais, Baterias e Percussão, ...",[],False,0.00,399,399.00,2022-04-19,None,Instrumentos Musicais
8971,Caixa Selada Lateral De Fibra Renault Sandero ...,"[Acessórios para Veículos, Som Automotivo, Cai...",[],False,4.72,308,293.46,2022-04-19,Som Automotivo,Acessórios para Veículos
9135,Pack Energético Red Bull Lata 6 Unidades 250ml...,"[Alimentos e Bebidas, Bebidas, Bebidas Energét...","[MAIS VENDIDO, 6º em Bebidas Energéticas]",False,4.68,44,41.94,2022-04-20,Bebidas Energéticas,Alimentos e Bebidas
